In [0]:
# Load Packages, set storage links |

from pyspark.sql import functions as f
from pyspark.sql import Window
from pyspark.sql.functions import col, to_timestamp, to_utc_timestamp, concat_ws, udf
from datetime import datetime

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# import airporttime

# Set Blob paths, create links | 

blob_container = "w261-team28-container" # The name of your container created in https://portal.azure.com
storage_account = "team28" # The name of your Storage account created in https://portal.azure.com
secret_scope = "w261-team28-scope" # The name of the scope created in your local computer using the Databricks CLI
secret_key = "w261-team28-key" # The name of the secret key created in your local computer using the Databricks CLI 
blob_url = f"wasbs://{blob_container}@{storage_account}.blob.core.windows.net"

spark.conf.set(
  f"fs.azure.sas.{blob_container}.{storage_account}.blob.core.windows.net",
  dbutils.secrets.get(scope = secret_scope, key = secret_key)
)

#### Load Data

In [0]:
# Load data from Saved Parquet Files | 

flights_3m = spark.read.parquet(f'{blob_url}/3m_flights_processed').cache()
flights_6m = spark.read.parquet(f'{blob_url}/6m_flights_processed').cache()
ICAO_station_mapping = spark.read.parquet(f'{blob_url}/ICAO_station_mapping').cache()
weather_sample_processed = spark.read.parquet(f'{blob_url}/30k_weather_sample_processed').cache()
weather_sample_processed_new = spark.read.parquet(f'{blob_url}/weather_2015_02_21').cache()

# Quick rename for sake of brevity
flights = flights_3m
stations = ICAO_station_mapping
weather = weather_sample_processed_new

In [0]:
display(weather_sample_processed)

STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,HOUR,WND_DIRECTION_ANGLE,WND_TYPE_CODE,WND_SPEED_RATE,CIG_CEILING_HEIGHT_DIMENSION,CIG_CEILING_DETERMINATION_CODE,CIG_CAVOK_CODE,VIS_DISTANCE_DIMENSION,VIS_VARIABILITY_CODE,TMP_AIR_TEMP,DEW_POINT_TEMP,SLP_SEA_LEVEL_PRES,VALID_WEATHER_DATA,WND_DIRECTION_ANGLE-AVG,WND_SPEED_RATE-AVG,CIG_CEILING_HEIGHT_DIMENSION-AVG,VIS_DISTANCE_DIMENSION-AVG,TMP_AIR_TEMP-AVG,DEW_POINT_TEMP-AVG,SLP_SEA_LEVEL_PRES-AVG
null,2015-02-01T12:15:00.000+0000,38.25,-122.6,27.1,"PETALUMA MUNICIPAL AIRPORT, CA US",2015-02-01T12:00:00.000+0000,null,C,0,91,M,N,16093,N,40,40,null,0,null,0.0,91.0,16093.0,40.0,40.0,null
null,2015-02-08T08:35:00.000+0000,38.25,-122.6,27.1,"PETALUMA MUNICIPAL AIRPORT, CA US",2015-02-08T08:00:00.000+0000,null,C,0,671,M,N,16093,N,140,130,null,0,null,0.0,671.0,16093.0,140.0,130.0,null
null,2015-02-22T22:55:00.000+0000,38.25,-122.6,27.1,"PETALUMA MUNICIPAL AIRPORT, CA US",2015-02-22T22:00:00.000+0000,50,N,36,22000,9,N,16093,N,180,-10,null,0,50.0,36.0,22000.0,16093.0,180.0,-10.0,null
null,2015-03-20T11:15:00.000+0000,38.25,-122.6,27.1,"PETALUMA MUNICIPAL AIRPORT, CA US",2015-03-20T11:00:00.000+0000,null,C,0,22000,9,N,14484,N,60,60,null,0,null,0.0,22000.0,14484.0,60.0,60.0,null
69015093121,2015-02-18T22:56:00.000+0000,34.3,-116.16667,625.1,"TWENTYNINE PALMS MC, CA US",2015-02-18T22:00:00.000+0000,null,C,0,22000,9,N,16093,N,244,-44,9960,0,null,0.0,22000.0,16093.0,244.0,-44.0,9960.0
69015093121,2015-02-26T14:56:00.000+0000,34.3,-116.16667,625.1,"TWENTYNINE PALMS MC, CA US",2015-02-26T14:00:00.000+0000,290,N,41,22000,9,N,16093,N,78,-28,10135,1,290.0,41.0,22000.0,16093.0,78.0,-28.0,10135.0
69015093121,2015-03-19T11:56:00.000+0000,34.3,-116.16667,625.1,"TWENTYNINE PALMS MC, CA US",2015-03-19T11:00:00.000+0000,270,N,31,22000,9,N,16093,N,156,50,10141,1,270.0,31.0,22000.0,16093.0,156.0,50.0,10141.0
69015093121,2015-03-19T16:56:00.000+0000,34.3,-116.16667,625.1,"TWENTYNINE PALMS MC, CA US",2015-03-19T16:00:00.000+0000,350,N,21,22000,9,N,16093,N,189,56,10160,1,350.0,21.0,22000.0,16093.0,189.0,56.0,10160.0
69015093121,2015-03-31T14:56:00.000+0000,34.3,-116.16667,625.1,"TWENTYNINE PALMS MC, CA US",2015-03-31T14:00:00.000+0000,null,V,15,22000,9,N,16093,N,178,-11,10140,0,null,15.0,22000.0,16093.0,178.0,-11.0,10140.0
70000126492,2015-01-29T15:53:00.000+0000,60.785,-148.83889,31.4,"PORTAGE GLACIER VISITOR CENTER, AK US",2015-01-29T15:00:00.000+0000,null,C,0,1676,M,N,6437,N,-89,-94,10160,0,null,0.0,1676.0,6437.0,-89.0,-94.0,10160.0


#### Schema Checks

In [0]:
# Schema Checks | 

flights.printSchema()
stations.printSchema()
weather.printSchema()

root
-- YEAR: integer (nullable = true)
-- QUARTER: integer (nullable = true)
-- MONTH: integer (nullable = true)
-- DAY_OF_MONTH: integer (nullable = true)
-- DAY_OF_WEEK: integer (nullable = true)
-- FL_DATE: string (nullable = true)
-- OP_UNIQUE_CARRIER: string (nullable = true)
-- OP_CARRIER_AIRLINE_ID: integer (nullable = true)
-- OP_CARRIER: string (nullable = true)
-- TAIL_NUM: string (nullable = true)
-- OP_CARRIER_FL_NUM: integer (nullable = true)
-- ORIGIN_AIRPORT_ID: integer (nullable = true)
-- ORIGIN_AIRPORT_SEQ_ID: integer (nullable = true)
-- ORIGIN_CITY_MARKET_ID: integer (nullable = true)
-- ORIGIN: string (nullable = true)
-- ORIGIN_CITY_NAME: string (nullable = true)
-- ORIGIN_STATE_ABR: string (nullable = true)
-- ORIGIN_STATE_FIPS: integer (nullable = true)
-- ORIGIN_STATE_NM: string (nullable = true)
-- ORIGIN_WAC: integer (nullable = true)
-- DEST_AIRPORT_ID: integer (nullable = true)
-- DEST_AIRPORT_SEQ_ID: integer (nullable = true)
-- DEST_CITY_MARKET_ID: integer (nullable = true)
-- DEST: string (nullable = true)
-- DEST_CITY_NAME: string (nullable = true)
-- DEST_STATE_ABR: string (nullable = true)
-- DEST_STATE_FIPS: integer (nullable = true)
-- DEST_STATE_NM: string (nullable = true)
-- DEST_WAC: integer (nullable = true)
-- CRS_DEP_TIME: integer (nullable = true)
-- DEP_DEL15: double (nullable = true)
-- DEP_DELAY_GROUP: integer (nullable = true)
-- DEP_TIME_BLK: string (nullable = true)
-- TAXI_OUT: double (nullable = true)
-- WHEELS_OFF: integer (nullable = true)
-- WHEELS_ON: integer (nullable = true)
-- TAXI_IN: double (nullable = true)
-- CRS_ARR_TIME: integer (nullable = true)
-- CANCELLED: double (nullable = true)
-- DIVERTED: double (nullable = true)
-- CRS_ELAPSED_TIME: double (nullable = true)
-- AIR_TIME: double (nullable = true)
-- FLIGHTS: double (nullable = true)
-- DISTANCE: double (nullable = true)
-- DISTANCE_GROUP: integer (nullable = true)
-- ID: string (nullable = true)
-- IATA_ORIGIN: string (nullable = true)
-- ICAO_ORIGIN: string (nullable = true)
-- AIRPORT_LAT_ORIGIN: double (nullable = true)
-- AIRPORT_LONG_ORIGIN: double (nullable = true)
-- AIRPORT_TIMEZONE_ORIGIN: string (nullable = true)
-- AIRPORT_UTC_OFFSET_ORIGIN: string (nullable = true)
-- IATA_DEST: string (nullable = true)
-- ICAO_DEST: string (nullable = true)
-- AIRPORT_LAT_DEST: double (nullable = true)
-- AIRPORT_LONG_DEST: double (nullable = true)
-- AIRPORT_TIMEZONE_DEST: string (nullable = true)
-- AIRPORT_UTC_OFFSET_DEST: string (nullable = true)
-- CRS_DEP_TIME_UTC: timestamp (nullable = true)
-- CRS_ARR_TIME_UTC: timestamp (nullable = true)
-- CRS_DEP_TIME_UTC_HOUR: timestamp (nullable = true)
-- CRS_ARR_TIME_UTC_HOUR: timestamp (nullable = true)

root
-- ICAO: string (nullable = true)
-- WEATHER_STATION_ID: string (nullable = true)

root
-- STATION: string (nullable = true)
-- DATE: timestamp (nullable = true)
-- LATITUDE: double (nullable = true)
-- LONGITUDE: double (nullable = true)
-- ELEVATION: double (nullable = true)
-- NAME: string (nullable = true)
-- HOUR: timestamp (nullable = true)
-- WND_DIRECTION_ANGLE: integer (nullable = true)
-- WND_TYPE_CODE: string (nullable = true)
-- WND_SPEED_RATE: integer (nullable = true)
-- CIG_CEILING_HEIGHT_DIMENSION: integer (nullable = true)
-- CIG_CEILING_DETERMINATION_CODE: string (nullable = true)
-- CIG_CAVOK_CODE: string (nullable = true)
-- VIS_DISTANCE_DIMENSION: integer (nullable = true)
-- VIS_VARIABILITY_CODE: string (nullable = true)
-- TMP_AIR_TEMP: integer (nullable = true)
-- DEW_POINT_TEMP: integer (nullable = true)
-- SLP_SEA_LEVEL_PRES: integer (nullable = true)
-- VALID_WEATHER_DATA: integer (nullable = true)
-- WND_DIRECTION_ANGLE-AVG: double (nullable = true)
-- WND_SPEED_RATE-AVG: double (nullable = true)
-- CIG_CEILING_HEIGHT_DIMENSION-AVG: double (nullable = true)
-- VIS_DISTANCE_DIMENSION-AVG: double (nullable = true)
-- TMP_AIR_TEMP-AVG: double (nullable = true)
-- DEW_POINT_TEMP-AVG: double (nullable = true)
-- SLP_SEA_LEVEL_PRES

In [0]:
# First, join flights to stations on ICAO - airport code which is a unique identifier we can use for both origin and destination
# In this join, we will be including:
#    - `WEATHER_STATION_ID`: the closest weather station which will then allow us to pull in weather at each airport

flights_w_stations = flights.join(stations, flights.ICAO_ORIGIN == stations.ICAO, 'left')\
                            .select(flights['*'], stations['WEATHER_STATION_ID'].alias('ORIGIN_WEATHER_STATION_ID'))\
                            .join(stations, flights.ICAO_DEST == stations.ICAO, 'left')\
                            .select(flights['*'], 'ORIGIN_WEATHER_STATION_ID', stations['WEATHER_STATION_ID'].alias('DEST_WEATHER_STATION_ID'))

In [0]:
# Sanity Check
# flights_w_stations.select(['ORIGIN', 'DEST', 'ICAO_ORIGIN', 'ICAO_DEST', 'ORIGIN_WEATHER_STATION_ID', 'DEST_WEATHER_STATION_ID']).distinct().where(f.col('ICAO_DEST').isin(['KORD', 'KATL', 'KFSD', 'KROC', 'KTPA', 'KGPI', 'KOAK', 'KRAP'])).display()
# stations.where(f.col('ICAO').isin(['KORD', 'KATL', 'KFSD', 'KROC', 'KTPA', 'KGPI', 'KOAK', 'KRAP'])).display()
flights_w_stations.printSchema()

root
-- YEAR: integer (nullable = true)
-- QUARTER: integer (nullable = true)
-- MONTH: integer (nullable = true)
-- DAY_OF_MONTH: integer (nullable = true)
-- DAY_OF_WEEK: integer (nullable = true)
-- FL_DATE: string (nullable = true)
-- OP_UNIQUE_CARRIER: string (nullable = true)
-- OP_CARRIER_AIRLINE_ID: integer (nullable = true)
-- OP_CARRIER: string (nullable = true)
-- TAIL_NUM: string (nullable = true)
-- OP_CARRIER_FL_NUM: integer (nullable = true)
-- ORIGIN_AIRPORT_ID: integer (nullable = true)
-- ORIGIN_AIRPORT_SEQ_ID: integer (nullable = true)
-- ORIGIN_CITY_MARKET_ID: integer (nullable = true)
-- ORIGIN: string (nullable = true)
-- ORIGIN_CITY_NAME: string (nullable = true)
-- ORIGIN_STATE_ABR: string (nullable = true)
-- ORIGIN_STATE_FIPS: integer (nullable = true)
-- ORIGIN_STATE_NM: string (nullable = true)
-- ORIGIN_WAC: integer (nullable = true)
-- DEST_AIRPORT_ID: integer (nullable = true)
-- DEST_AIRPORT_SEQ_ID: integer (nullable = true)
-- DEST_CITY_MARKET_ID: integer (nullable = true)
-- DEST: string (nullable = true)
-- DEST_CITY_NAME: string (nullable = true)
-- DEST_STATE_ABR: string (nullable = true)
-- DEST_STATE_FIPS: integer (nullable = true)
-- DEST_STATE_NM: string (nullable = true)
-- DEST_WAC: integer (nullable = true)
-- CRS_DEP_TIME: integer (nullable = true)
-- DEP_DEL15: double (nullable = true)
-- DEP_DELAY_GROUP: integer (nullable = true)
-- DEP_TIME_BLK: string (nullable = true)
-- TAXI_OUT: double (nullable = true)
-- WHEELS_OFF: integer (nullable = true)
-- WHEELS_ON: integer (nullable = true)
-- TAXI_IN: double (nullable = true)
-- CRS_ARR_TIME: integer (nullable = true)
-- CANCELLED: double (nullable = true)
-- DIVERTED: double (nullable = true)
-- CRS_ELAPSED_TIME: double (nullable = true)
-- AIR_TIME: double (nullable = true)
-- FLIGHTS: double (nullable = true)
-- DISTANCE: double (nullable = true)
-- DISTANCE_GROUP: integer (nullable = true)
-- ID: string (nullable = true)
-- IATA_ORIGIN: string (nullable = true)
-- ICAO_ORIGIN: string (nullable = true)
-- AIRPORT_LAT_ORIGIN: double (nullable = true)
-- AIRPORT_LONG_ORIGIN: double (nullable = true)
-- AIRPORT_TIMEZONE_ORIGIN: string (nullable = true)
-- AIRPORT_UTC_OFFSET_ORIGIN: string (nullable = true)
-- IATA_DEST: string (nullable = true)
-- ICAO_DEST: string (nullable = true)
-- AIRPORT_LAT_DEST: double (nullable = true)
-- AIRPORT_LONG_DEST: double (nullable = true)
-- AIRPORT_TIMEZONE_DEST: string (nullable = true)
-- AIRPORT_UTC_OFFSET_DEST: string (nullable = true)
-- CRS_DEP_TIME_UTC: timestamp (nullable = true)
-- CRS_ARR_TIME_UTC: timestamp (nullable = true)
-- CRS_DEP_TIME_UTC_HOUR: timestamp (nullable = true)
-- CRS_ARR_TIME_UTC_HOUR: timestamp (nullable = true)
-- ORIGIN_WEATHER_STATION_ID: string (nullable = true)
-- DEST_WEATHER_STATION_ID: string (nullable = true)

In [0]:
# Check to make sure new timestamp mapping looks correct
# flights_w_stations.select('CRS_DEP_TIME_UTC_HOUR').withColumn('new_time', to_timestamp(f.col('CRS_DEP_TIME_UTC_HOUR').cast('long') - 10800)).display()

CRS_DEP_TIME_UTC_HOUR,new_time
2015-02-01T13:00:00.000+0000,2015-02-01T10:00:00.000+0000
2015-02-03T13:00:00.000+0000,2015-02-03T10:00:00.000+0000
2015-02-04T13:00:00.000+0000,2015-02-04T10:00:00.000+0000
2015-02-05T13:00:00.000+0000,2015-02-05T10:00:00.000+0000
2015-02-06T13:00:00.000+0000,2015-02-06T10:00:00.000+0000
2015-02-08T13:00:00.000+0000,2015-02-08T10:00:00.000+0000
2015-02-09T13:00:00.000+0000,2015-02-09T10:00:00.000+0000
2015-02-10T13:00:00.000+0000,2015-02-10T10:00:00.000+0000
2015-02-11T13:00:00.000+0000,2015-02-11T10:00:00.000+0000
2015-02-12T13:00:00.000+0000,2015-02-12T10:00:00.000+0000


In [0]:
# Second, we need to create a new timestamp to join weather on - which is lagged by 2 hours. 
# Due to the nature of our timestamps - we will actually have to subtract 3 hours to avoid leakage. 10800 is equivalent to 3 hours
flights_w_stations = flights_w_stations.withColumn('CRS_DEP_TIME_UTC_LAG', to_timestamp(f.col('CRS_DEP_TIME_UTC_HOUR').cast('long') - 10800))

# Next, prepend an origin or destination prefix to the weather columns, so when we join we know which weather set we're looking at
origin_weather = weather.select([f.col(weather_feat).alias('ORIGIN_WEATHER_'+weather_feat) for weather_feat in weather.columns])
dest_weather = weather.select([f.col(weather_feat).alias('DEST_WEATHER_'+weather_feat) for weather_feat in weather.columns])

# join flights to ORIGIN weather on station_id and our lagged ORIGIN time variable
flights_w_weather_temp = flights_w_stations.join(origin_weather, (flights_w_stations.ORIGIN_WEATHER_STATION_ID == origin_weather.ORIGIN_WEATHER_STATION) &\
                                                          (flights_w_stations.CRS_DEP_TIME_UTC_LAG == origin_weather.ORIGIN_WEATHER_HOUR), 
                                                          'left')

# Finally, join flights to DESTINATION weather on station_id and lagged ORIGIN time variable
flights_w_weather = flights_w_weather_temp.join(dest_weather, (flights_w_weather_temp.DEST_WEATHER_STATION_ID == dest_weather.DEST_WEATHER_STATION) &\
                                                          (flights_w_weather_temp.CRS_DEP_TIME_UTC_LAG == dest_weather.DEST_WEATHER_HOUR), 
                                                          'left')

# flights_w_weather_temp.count()
# flights_w_weather_temp.where(f.col('ID') == '2015-02-01N3MEAAORD730').display()

In [0]:
# flights_w_weather_temp.where(f.col('FL_DATE') == '2015-02-21').display()
# flights_w_weather_temp.where(f.col('ID') == '2015-02-21N4XDAAORD1805').display()
flights_w_weather.where(f.col('ID') == '2015-02-21N4XDAAORD1805').display()

YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_WAC,CRS_DEP_TIME,DEP_DEL15,DEP_DELAY_GROUP,DEP_TIME_BLK,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,CANCELLED,DIVERTED,CRS_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,ID,IATA_ORIGIN,ICAO_ORIGIN,AIRPORT_LAT_ORIGIN,AIRPORT_LONG_ORIGIN,AIRPORT_TIMEZONE_ORIGIN,AIRPORT_UTC_OFFSET_ORIGIN,IATA_DEST,ICAO_DEST,AIRPORT_LAT_DEST,AIRPORT_LONG_DEST,AIRPORT_TIMEZONE_DEST,AIRPORT_UTC_OFFSET_DEST,CRS_DEP_TIME_UTC,CRS_ARR_TIME_UTC,CRS_DEP_TIME_UTC_HOUR,CRS_ARR_TIME_UTC_HOUR,ORIGIN_WEATHER_STATION_ID,DEST_WEATHER_STATION_ID,CRS_DEP_TIME_UTC_LAG,ORIGIN_WEATHER_STATION,ORIGIN_WEATHER_DATE,ORIGIN_WEATHER_LATITUDE,ORIGIN_WEATHER_LONGITUDE,ORIGIN_WEATHER_ELEVATION,ORIGIN_WEATHER_NAME,ORIGIN_WEATHER_HOUR,ORIGIN_WEATHER_WND_DIRECTION_ANGLE,ORIGIN_WEATHER_WND_TYPE_CODE,ORIGIN_WEATHER_WND_SPEED_RATE,ORIGIN_WEATHER_CIG_CEILING_HEIGHT_DIMENSION,ORIGIN_WEATHER_CIG_CEILING_DETERMINATION_CODE,ORIGIN_WEATHER_CIG_CAVOK_CODE,ORIGIN_WEATHER_VIS_DISTANCE_DIMENSION,ORIGIN_WEATHER_VIS_VARIABILITY_CODE,ORIGIN_WEATHER_TMP_AIR_TEMP,ORIGIN_WEATHER_DEW_POINT_TEMP,ORIGIN_WEATHER_SLP_SEA_LEVEL_PRES,ORIGIN_WEATHER_VALID_WEATHER_DATA,ORIGIN_WEATHER_WND_DIRECTION_ANGLE-AVG,ORIGIN_WEATHER_WND_SPEED_RATE-AVG,ORIGIN_WEATHER_CIG_CEILING_HEIGHT_DIMENSION-AVG,ORIGIN_WEATHER_VIS_DISTANCE_DIMENSION-AVG,ORIGIN_WEATHER_TMP_AIR_TEMP-AVG,ORIGIN_WEATHER_DEW_POINT_TEMP-AVG,ORIGIN_WEATHER_SLP_SEA_LEVEL_PRES-AVG,DEST_WEATHER_STATION,DEST_WEATHER_DATE,DEST_WEATHER_LATITUDE,DEST_WEATHER_LONGITUDE,DEST_WEATHER_ELEVATION,DEST_WEATHER_NAME,DEST_WEATHER_HOUR,DEST_WEATHER_WND_DIRECTION_ANGLE,DEST_WEATHER_WND_TYPE_CODE,DEST_WEATHER_WND_SPEED_RATE,DEST_WEATHER_CIG_CEILING_HEIGHT_DIMENSION,DEST_WEATHER_CIG_CEILING_DETERMINATION_CODE,DEST_WEATHER_CIG_CAVOK_CODE,DEST_WEATHER_VIS_DISTANCE_DIMENSION,DEST_WEATHER_VIS_VARIABILITY_CODE,DEST_WEATHER_TMP_AIR_TEMP,DEST_WEATHER_DEW_POINT_TEMP,DEST_WEATHER_SLP_SEA_LEVEL_PRES,DEST_WEATHER_VALID_WEATHER_DATA,DEST_WEATHER_WND_DIRECTION_ANGLE-AVG,DEST_WEATHER_WND_SPEED_RATE-AVG,DEST_WEATHER_CIG_CEILING_HEIGHT_DIMENSION-AVG,DEST_WEATHER_VIS_DISTANCE_DIMENSION-AVG,DEST_WEATHER_TMP_AIR_TEMP-AVG,DEST_WEATHER_DEW_POINT_TEMP-AVG,DEST_WEATHER_SLP_SEA_LEVEL_PRES-AVG
2015,1,2,21,6,2015-02-21,AA,19805,AA,N4XDAA,2409,13930,1393003,30977,ORD,"Chicago, IL",IL,17,Illinois,41,15304,1530402,33195,TPA,"Tampa, FL",FL,12,Florida,33,1805,0.0,-1,1800-1859,14.0,1816,2128,4.0,2140,0.0,0.0,155.0,132.0,1.0,1012.0,5,2015-02-21N4XDAAORD1805,ORD,KORD,41.9786,-87.9048,America/Chicago,-6,TPA,KTPA,27.975500106811523,-82.533203125,America/New_York,-5,2015-02-22T00:05:00.000+0000,2015-02-22T02:40:00.000+0000,2015-02-22T00:00:00.000+0000,2015-02-22T02:00:00.000+0000,72530094846,72211012842,2015-02-21T21:00:00.000+0000,72530094846,2015-02-21T21:51:00.000+0000,41.995,-87.9336,201.8,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",2015-02-21T21:00:00.000+0000,null,C,0,884,M,N,12875,N,-11,-78,10154,0,200.0,0.0,884.0,12875.0,-11.0,-78.0,10154.0,72211012842,2015-02-21T21:53:00.000+0000,27.96194,-82.5403,5.8,"TAMPA INTERNATIONAL AIRPORT, FL US",2015-02-21T21:00:00.000+0000,140,N,26,22000,9,N,16093,N,228,89,10224,1,140.0,26.0,22000.0,16093.0,228.0,89.0,10224.0


156227
156227

In [0]:
flights_w_weather_temp.write.parquet(f'{blob_url}/data_for_aditya')

In [0]:
weather.where(((f.col('DATE') > "2015-02-20T23:59:59.000") & (f.col('DATE') < "2015-02-22T00:00:00.000")) & (f.col('STATION') == '72211012842')).display() 

STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,HOUR,WND_DIRECTION_ANGLE,WND_TYPE_CODE,WND_SPEED_RATE,CIG_CEILING_HEIGHT_DIMENSION,CIG_CEILING_DETERMINATION_CODE,CIG_CAVOK_CODE,VIS_DISTANCE_DIMENSION,VIS_VARIABILITY_CODE,TMP_AIR_TEMP,DEW_POINT_TEMP,SLP_SEA_LEVEL_PRES,VALID_WEATHER_DATA,WND_DIRECTION_ANGLE-AVG,WND_SPEED_RATE-AVG,CIG_CEILING_HEIGHT_DIMENSION-AVG,VIS_DISTANCE_DIMENSION-AVG,TMP_AIR_TEMP-AVG,DEW_POINT_TEMP-AVG,SLP_SEA_LEVEL_PRES-AVG
72211012842,2015-02-21T00:53:00.000+0000,27.96194,-82.5403,5.8,"TAMPA INTERNATIONAL AIRPORT, FL US",2015-02-21T00:00:00.000+0000,60,N,31,22000,9,N,16093,N,111,-78,10281,1,60.0,31.0,22000.0,16093.0,111.0,-78.0,10281.0
72211012842,2015-02-21T01:53:00.000+0000,27.96194,-82.5403,5.8,"TAMPA INTERNATIONAL AIRPORT, FL US",2015-02-21T01:00:00.000+0000,40,N,26,22000,9,N,16093,N,100,-78,10287,1,40.0,26.0,22000.0,16093.0,100.0,-78.0,10287.0
72211012842,2015-02-21T02:53:00.000+0000,27.96194,-82.5403,5.8,"TAMPA INTERNATIONAL AIRPORT, FL US",2015-02-21T02:00:00.000+0000,70,N,31,22000,9,N,16093,N,94,-61,10295,1,70.0,31.0,22000.0,16093.0,94.0,-61.0,10295.0
72211012842,2015-02-21T03:53:00.000+0000,27.96194,-82.5403,5.8,"TAMPA INTERNATIONAL AIRPORT, FL US",2015-02-21T03:00:00.000+0000,70,N,26,22000,9,N,16093,N,83,-11,10295,1,70.0,26.0,22000.0,16093.0,83.0,-11.0,10295.0
72211012842,2015-02-21T04:59:00.000+0000,27.96194,-82.5403,5.8,"TAMPA INTERNATIONAL AIRPORT, FL US",2015-02-21T04:00:00.000+0000,null,null,null,null,9,null,null,null,null,null,null,0,60.0,28.5,22000.0,16093.0,97.0,-57.0,10289.5
72211012842,2015-02-21T05:53:00.000+0000,27.96194,-82.5403,5.8,"TAMPA INTERNATIONAL AIRPORT, FL US",2015-02-21T05:00:00.000+0000,70,N,41,22000,9,N,16093,N,72,11,10290,1,70.0,41.0,22000.0,16093.0,72.0,11.0,10290.0
72211012842,2015-02-21T06:53:00.000+0000,27.96194,-82.5403,5.8,"TAMPA INTERNATIONAL AIRPORT, FL US",2015-02-21T06:00:00.000+0000,70,N,21,22000,9,N,16093,N,67,11,10283,1,70.0,21.0,22000.0,16093.0,67.0,11.0,10283.0
72211012842,2015-02-21T07:53:00.000+0000,27.96194,-82.5403,5.8,"TAMPA INTERNATIONAL AIRPORT, FL US",2015-02-21T07:00:00.000+0000,null,V,21,22000,9,N,16093,N,67,11,10275,0,63.333333333333336,21.0,22000.0,16093.0,67.0,11.0,10275.0
72211012842,2015-02-21T08:53:00.000+0000,27.96194,-82.5403,5.8,"TAMPA INTERNATIONAL AIRPORT, FL US",2015-02-21T08:00:00.000+0000,60,N,21,22000,9,N,16093,N,67,17,10270,1,60.0,21.0,22000.0,16093.0,67.0,17.0,10270.0
72211012842,2015-02-21T09:53:00.000+0000,27.96194,-82.5403,5.8,"TAMPA INTERNATIONAL AIRPORT, FL US",2015-02-21T09:00:00.000+0000,70,N,26,22000,9,N,16093,N,67,17,10265,1,70.0,26.0,22000.0,16093.0,67.0,17.0,10265.0


In [0]:
weather.printSchema()

root
-- STATION: string (nullable = true)
-- DATE: timestamp (nullable = true)
-- LATITUDE: double (nullable = true)
-- LONGITUDE: double (nullable = true)
-- ELEVATION: double (nullable = true)
-- NAME: string (nullable = true)
-- HOUR: timestamp (nullable = true)
-- WND_DIRECTION_ANGLE: integer (nullable = true)
-- WND_TYPE_CODE: string (nullable = true)
-- WND_SPEED_RATE: integer (nullable = true)
-- CIG_CEILING_HEIGHT_DIMENSION: integer (nullable = true)
-- CIG_CEILING_DETERMINATION_CODE: string (nullable = true)
-- CIG_CAVOK_CODE: string (nullable = true)
-- VIS_DISTANCE_DIMENSION: integer (nullable = true)
-- VIS_VARIABILITY_CODE: string (nullable = true)
-- TMP_AIR_TEMP: integer (nullable = true)
-- DEW_POINT_TEMP: integer (nullable = true)
-- SLP_SEA_LEVEL_PRES: integer (nullable = true)
-- VALID_WEATHER_DATA: integer (nullable = true)
-- WND_DIRECTION_ANGLE-AVG: double (nullable = true)
-- WND_SPEED_RATE-AVG: double (nullable = true)
-- CIG_CEILING_HEIGHT_DIMENSION-AVG: double (nullable = true)
-- VIS_DISTANCE_DIMENSION-AVG: double (nullable = true)
-- TMP_AIR_TEMP-AVG: double (nullable = true)
-- DEW_POINT_TEMP-AVG: double (nullable = true)
-- SLP_SEA_LEVEL_PRES-AVG: double (nullable = true)

In [0]:
stations2 = stations
prefix = 'ORIGIN'
stations3 = stations2.select([col(c).alias(prefix+c) for c in stations2.columns])
stations3.display()

In [0]:
flights_w_weather_temp2 = flights_w_stations.join(weather, (flights_w_stations.ORIGIN_WEATHER_STATION_ID == weather.STATION) &\
                                                          (flights_w_stations.CRS_DEP_TIME_UTC_HOUR == weather.HOUR))

flights_w_weather_temp2.display()

YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_WAC,CRS_DEP_TIME,DEP_DEL15,DEP_DELAY_GROUP,DEP_TIME_BLK,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,CANCELLED,DIVERTED,CRS_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,ID,IATA_ORIGIN,ICAO_ORIGIN,AIRPORT_LAT_ORIGIN,AIRPORT_LONG_ORIGIN,AIRPORT_TIMEZONE_ORIGIN,AIRPORT_UTC_OFFSET_ORIGIN,IATA_DEST,ICAO_DEST,AIRPORT_LAT_DEST,AIRPORT_LONG_DEST,AIRPORT_TIMEZONE_DEST,AIRPORT_UTC_OFFSET_DEST,CRS_DEP_TIME_UTC,CRS_ARR_TIME_UTC,CRS_DEP_TIME_UTC_HOUR,CRS_ARR_TIME_UTC_HOUR,ORIGIN_WEATHER_STATION_ID,DEST_WEATHER_STATION_ID,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,HOUR,WND_DIRECTION_ANGLE,WND_TYPE_CODE,WND_SPEED_RATE,CIG_CEILING_HEIGHT_DIMENSION,CIG_CEILING_DETERMINATION_CODE,CIG_CAVOK_CODE,VIS_DISTANCE_DIMENSION,VIS_VARIABILITY_CODE,TMP_AIR_TEMP,DEW_POINT_TEMP,SLP_SEA_LEVEL_PRES,VALID_WEATHER_DATA,WND_DIRECTION_ANGLE-AVG,WND_SPEED_RATE-AVG,CIG_CEILING_HEIGHT_DIMENSION-AVG,VIS_DISTANCE_DIMENSION-AVG,TMP_AIR_TEMP-AVG,DEW_POINT_TEMP-AVG,SLP_SEA_LEVEL_PRES-AVG
2015,1,2,12,4,2015-02-12,AA,19805,AA,N4YRAA,1065,13930,1393003,30977,ORD,"Chicago, IL",IL,17,Illinois,41,10721,1072102,30721,BOS,"Boston, MA",MA,25,Massachusetts,13,1300,0.0,0,1300-1359,16.0,1324,1616,6.0,1613,0.0,0.0,133.0,112.0,1.0,867.0,4,2015-02-12N4YRAAORD1300,ORD,KORD,41.9786,-87.9048,America/Chicago,-6,BOS,KBOS,42.36429977,-71.00520325,America/New_York,-5,2015-02-12T19:00:00.000+0000,2015-02-12T21:13:00.000+0000,2015-02-12T19:00:00.000+0000,2015-02-12T21:00:00.000+0000,72530094846,72509014739,72530094846,2015-02-12T19:51:00.000+0000,41.995,-87.9336,201.8,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",2015-02-12T19:00:00.000+0000,350,N,46,22000,9,N,16093,N,-89,-183,10318,1,350.0,46.0,22000.0,16093.0,-89.0,-183.0,10318.0
2015,1,2,12,4,2015-02-12,AA,19805,AA,N3MEAA,1644,13930,1393003,30977,ORD,"Chicago, IL",IL,17,Illinois,41,14908,1490803,32575,SNA,"Santa Ana, CA",CA,6,California,91,1345,1.0,1,1300-1359,13.0,1421,1553,7.0,1608,0.0,0.0,263.0,212.0,1.0,1726.0,7,2015-02-12N3MEAAORD1345,ORD,KORD,41.9786,-87.9048,America/Chicago,-6,SNA,KSNA,33.67570114,-117.8679962,America/Los_Angeles,-8,2015-02-12T19:45:00.000+0000,2015-02-13T00:08:00.000+0000,2015-02-12T19:00:00.000+0000,2015-02-13T00:00:00.000+0000,72530094846,72297793184,72530094846,2015-02-12T19:51:00.000+0000,41.995,-87.9336,201.8,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",2015-02-12T19:00:00.000+0000,350,N,46,22000,9,N,16093,N,-89,-183,10318,1,350.0,46.0,22000.0,16093.0,-89.0,-183.0,10318.0
2015,1,2,12,4,2015-02-12,MQ,20398,MQ,N650MQ,2941,13930,1393003,30977,ORD,"Chicago, IL",IL,17,Illinois,41,11267,1126702,31267,DAY,"Dayton, OH",OH,39,Ohio,44,1305,0.0,-1,1300-1359,10.0,1310,1456,3.0,1509,0.0,0.0,64.0,46.0,1.0,240.0,1,2015-02-12N650MQORD1305,ORD,KORD,41.9786,-87.9048,America/Chicago,-6,DAY,KDAY,39.90240097045898,-84.21939849853516,America/New_York,-5,2015-02-12T19:05:00.000+0000,2015-02-12T20:09:00.000+0000,2015-02-12T19:00:00.000+0000,2015-02-12T20:00:00.000+0000,72530094846,72429093815,72530094846,2015-02-12T19:51:00.000+0000,41.995,-87.9336,201.8,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",2015-02-12T19:00:00.000+0000,350,N,46,22000,9,N,16093,N,-89,-183,10318,1,350.0,46.0,22000.0,16093.0,-89.0,-183.0,10318.0
2015,1,2,12,4,2015-02-12,MQ,20398,MQ,N528MQ,3020,13930,1393003,30977,ORD,"Chicago, IL",IL,17,Illinois,41,11042,1104203,30647,CLE,"Cleveland, OH",OH,39,Ohio,44,1323,1.0,4,1300-1359,18.0,1448,1639,5.0,1535,0.0,0.0,72.0,51.0,1.0,315.0,2,2015-02-12N528MQORD1323,ORD,KORD,41.9786,-87.9048,America/Chicago,-6,CLE,KCLE,41.4117012024,-81.8498001099,America/New_York,-5,2015-02-12T19:23:00.000+0000,2015-02-12T20:35:00.00

In [0]:
weather.filter((col('DATE') > "2015-02-20T23:59:59.000") & (col('DATE') < "2015-02-22T00:00:00.000")).display()

STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,HOUR,WND_DIRECTION_ANGLE,WND_TYPE_CODE,WND_SPEED_RATE,CIG_CEILING_HEIGHT_DIMENSION,CIG_CEILING_DETERMINATION_CODE,CIG_CAVOK_CODE,VIS_DISTANCE_DIMENSION,VIS_VARIABILITY_CODE,TMP_AIR_TEMP,DEW_POINT_TEMP,SLP_SEA_LEVEL_PRES,VALID_WEATHER_DATA,WND_DIRECTION_ANGLE-AVG,WND_SPEED_RATE-AVG,CIG_CEILING_HEIGHT_DIMENSION-AVG,VIS_DISTANCE_DIMENSION-AVG,TMP_AIR_TEMP-AVG,DEW_POINT_TEMP-AVG,SLP_SEA_LEVEL_PRES-AVG
70000126492,2015-02-21T03:53:00.000+0000,60.785,-148.83889,31.4,"PORTAGE GLACIER VISITOR CENTER, AK US",2015-02-21T03:00:00.000+0000,100,N,82,1006,M,N,16093,N,56,17,10221,1,100.0,82.0,1006.0,16093.0,56.0,17.0,10221.0
70063899999,2015-02-21T09:01:00.000+0000,54.85,-163.4166666,6.0,"FALSE PASS, AK US",2015-02-21T09:00:00.000+0000,230,N,21,884,9,N,16093,null,30,30,null,0,230.0,21.0,884.0,16093.0,30.0,30.0,null
70148626642,2015-02-21T10:01:00.000+0000,67.73167,-164.54833,3.0,"KIVALINA AIRPORT, AK US",2015-02-21T10:00:00.000+0000,70,N,72,1097,M,N,16093,N,-17,-33,null,0,70.0,72.0,1097.0,16093.0,-17.0,-33.0,null
70204026703,2015-02-21T10:56:00.000+0000,63.76667,-171.73278,8.5,"GAMBELL AIRPORT, AK US",2015-02-21T10:00:00.000+0000,30,N,144,335,M,N,1207,N,-130,-150,null,0,30.0,144.0,335.0,1207.0,-130.0,-150.0,null
70222326602,2015-02-21T13:36:00.000+0000,64.935,-161.155,33.5,"KOYUK ALFRED ADAMS AIRPORT, AK US",2015-02-21T13:00:00.000+0000,30,N,57,2591,M,N,16093,N,null,null,null,0,30.0,57.0,2591.0,16093.0,null,null,null
70333325518,2015-02-21T18:56:00.000+0000,56.31139,-158.37306,5.5,"CHIGNIK AIRPORT, AK US",2015-02-21T18:00:00.000+0000,110,N,46,884,M,N,16093,N,60,20,null,0,110.0,46.0,884.0,16093.0,60.0,20.0,null
70350025501,2015-02-21T05:51:00.000+0000,57.75111,-152.48556,24.4,"KODIAK AIRPORT, AK US",2015-02-21T05:00:00.000+0000,50,N,72,152,M,N,4023,N,60,60,null,0,50.0,72.0,152.0,4023.0,60.0,60.0,null
70360525604,2015-02-21T20:56:00.000+0000,59.01139,-161.81972,4.6,"PLATINUM AIRPORT, AK US",2015-02-21T20:00:00.000+0000,110,N,82,488,M,N,4023,N,38,27,9963,1,110.0,82.0,488.0,4023.0,38.0,27.0,9963.0
70381025309,2015-02-21T06:00:00.000+0000,58.3566,-134.564,4.9,"JUNEAU AIRPORT, AK US",2015-02-21T06:00:00.000+0000,null,C,0,null,9,N,9000,null,0,-6,10308,0,70.0,0.0,1433.0,9000.0,0.0,-6.0,10308.0
70392500112,2015-02-21T18:56:00.000+0000,59.733,-157.267,82.3,"KOLIGANEK AIRPORT, AK US",2015-02-21T18:00:00.000+0000,130,N,154,1676,M,N,16093,N,38,5,10023,1,130.0,154.0,1676.0,16093.0,38.0,5.0,10023.0


In [0]:
flights.where(f.col('CRS_DEP_TIME_UTC_HOUR') == '2015-02-26T13:00:00.000+0000').distinct().display()

YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_WAC,CRS_DEP_TIME,DEP_DEL15,DEP_DELAY_GROUP,DEP_TIME_BLK,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,CANCELLED,DIVERTED,CRS_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,ID,IATA_ORIGIN,ICAO_ORIGIN,AIRPORT_LAT_ORIGIN,AIRPORT_LONG_ORIGIN,AIRPORT_TIMEZONE_ORIGIN,AIRPORT_UTC_OFFSET_ORIGIN,IATA_DEST,ICAO_DEST,AIRPORT_LAT_DEST,AIRPORT_LONG_DEST,AIRPORT_TIMEZONE_DEST,AIRPORT_UTC_OFFSET_DEST,CRS_DEP_TIME_UTC,CRS_ARR_TIME_UTC,CRS_DEP_TIME_UTC_HOUR,CRS_ARR_TIME_UTC_HOUR
2015,1,2,26,4,2015-02-26,OO,20304,OO,N809SK,4478,13930,1393003,30977,ORD,"Chicago, IL",IL,17,Illinois,41,14869,1486903,34614,SLC,"Salt Lake City, UT",UT,49,Utah,87,750,1.0,1,0700-0759,47.0,905,1056,8.0,1029,0.0,0.0,219.0,171.0,1.0,1250.0,6,2015-02-26N809SKORD750,ORD,KORD,41.9786,-87.9048,America/Chicago,-6,SLC,KSLC,40.78839874267578,-111.97799682617188,America/Denver,-7,2015-02-26T13:50:00.000+0000,2015-02-26T17:29:00.000+0000,2015-02-26T13:00:00.000+0000,2015-02-26T17:00:00.000+0000
2015,1,2,26,4,2015-02-26,MQ,20398,MQ,N611MQ,3392,13930,1393003,30977,ORD,"Chicago, IL",IL,17,Illinois,41,11042,1104203,30647,CLE,"Cleveland, OH",OH,39,Ohio,44,730,0.0,-1,0700-0759,52.0,814,1005,5.0,944,0.0,0.0,74.0,51.0,1.0,315.0,2,2015-02-26N611MQORD730,ORD,KORD,41.9786,-87.9048,America/Chicago,-6,CLE,KCLE,41.4117012024,-81.8498001099,America/New_York,-5,2015-02-26T13:30:00.000+0000,2015-02-26T14:44:00.000+0000,2015-02-26T13:00:00.000+0000,2015-02-26T14:00:00.000+0000
2015,1,2,26,4,2015-02-26,WN,19393,WN,N791SW,513,10397,1039705,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,15304,1530402,33195,TPA,"Tampa, FL",FL,12,Florida,33,850,0.0,-1,0800-0859,15.0,903,1007,6.0,1020,0.0,0.0,90.0,64.0,1.0,406.0,2,2015-02-26N791SWATL850,ATL,KATL,33.6367,-84.428101,America/New_York,-5,TPA,KTPA,27.975500106811523,-82.533203125,America/New_York,-5,2015-02-26T13:50:00.000+0000,2015-02-26T15:20:00.000+0000,2015-02-26T13:00:00.000+0000,2015-02-26T15:00:00.000+0000
2015,1,2,26,4,2015-02-26,UA,19977,UA,N38473,1168,13930,1393003,30977,ORD,"Chicago, IL",IL,17,Illinois,41,12892,1289203,32575,LAX,"Los Angeles, CA",CA,6,California,91,705,0.0,0,0700-0759,29.0,737,928,15.0,940,0.0,0.0,275.0,231.0,1.0,1744.0,7,2015-02-26N38473ORD705,ORD,KORD,41.9786,-87.9048,America/Chicago,-6,LAX,KLAX,33.94250107,-118.4079971,America/Los_Angeles,-8,2015-02-26T13:05:00.000+0000,2015-02-26T17:40:00.000+0000,2015-02-26T13:00:00.000+0000,2015-02-26T17:00:00.000+0000
2015,1,2,26,4,2015-02-26,EV,20366,EV,N873AS,5162,10397,1039705,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,11641,1164102,31641,FAY,"Fayetteville, NC",NC,37,North Carolina,36,820,0.0,-1,0800-0859,16.0,833,927,7.0,931,0.0,0.0,71.0,54.0,1.0,331.0,2,2015-02-26N873ASATL820,ATL,KATL,33.6367,-84.428101,America/New_York,-5,FAY,KFAY,34.9911994934082,-78.88030242919922,America/New_York,-5,2015-02-26T13:20:00.000+0000,2015-02-26T14:31:00.000+0000,2015-02-26T13:00:00.000+0000,2015-02-26T14:00:00.000+0000
2015,1,2,26,4,2015-02-26,WN,19393,WN,N631SW,1368,10397,1039705,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,13198,1319801,33198,MCI,"Kansas City, MO",MO,29,Missouri,64,850,0.0,-1,0800-0859,14.0,901,953,5.0,1010,0.0,0.0,140.0,112.0,1.0,692.0,3,2015-02-26N631SWATL850,ATL,KATL,33.6367,-84.428101,America/New_York,-5,MCI,KMCI,39.2976,-94.713898,America/Chicago,-6,2015-02-26T13:50:00.000+0000,2015-02-26T16:10:00.000+0000,2015-02-26T13:00:00.000+0000,2015-02-26T16:00:00.000+0000
2015,1,2,26,4,2015-02-26,DL,19790,DL,N6714Q,1294,10397,1039705,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,14492,1449202,34492,RDU,"Raleigh/Durham, NC",NC,37,North Carolina,36,859,1.0,4,0800-0859,15.0,1022,1110,8.0,1020,0.0,0.0,